In [2]:
import sys
import os
from pathlib import Path

# Add parent directory to path for imports
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

# Add this to your notebook
import importlib
import aicons.bayesbrainGPT.sensors.tf_sensors
# Reload the module to pick up our changes
importlib.reload(aicons.bayesbrainGPT.sensors.tf_sensors)

## Checking Action Space

In [3]:

# Step 2: Import our AIcon class
from aicons.definitions.simple_bad_aicon import SimpleBadAIcon

# Step 3: Create a new AIcon instance
aicon = SimpleBadAIcon(name="Test AIcon")

# Step 4: Add a continuous factor
aicon.add_factor_continuous(
    name="conversion_rate",
    value=0.05,
    uncertainty=0.01,
    lower_bound=0.0,
    upper_bound=1.0,
    description="Conversion rate for ads"
)

# Step 5: Verify the TensorFlow distribution was created
state_factors = aicon.brain.get_state_factors()
print("Factor created:", "conversion_rate" in state_factors)

# Step 6: Check if the factor has a TensorFlow distribution
factor = state_factors["conversion_rate"]
print("Has TF distribution:", "tf_distribution" in factor)

# Step 7: Get info about the distribution
if "tf_distribution" in factor:
    tf_dist = factor["tf_distribution"]
    dist_type = type(tf_dist).__name__
    print(f"Distribution type: {dist_type}")
    
    # Sample from the distribution
    samples = tf_dist.sample(5)
    print(f"5 samples from distribution: {samples.numpy()}")

Creating continuous factor with TensorFlow: conversion_rate
Added continuous factor with TensorFlow distribution: conversion_rate
Factor created: True
Has TF distribution: True
Distribution type: TruncatedNormal
5 samples from distribution: [0.04220393 0.04540392 0.05376558 0.04160205 0.06353349]


In [4]:
# Continue from previous code

# Step 8: Add a categorical factor
aicon.add_factor_categorical(
    name="best_channel",
    value="facebook",
    categories=["facebook", "google", "tiktok", "instagram"],
    probs=[0.4, 0.3, 0.2, 0.1],
    description="Best performing ad channel"
)

# Step 9: Verify categorical factor
cat_factor = aicon.brain.get_state_factors()["best_channel"]
print("\nCategorical factor:")
print(f"Distribution type: {type(cat_factor['tf_distribution']).__name__}")
print(f"Categories: {cat_factor['categories']}")
print(f"5 samples:", [aicon.brain.get_state_factors()["best_channel"]["categories"][int(i)] 
                    for i in cat_factor["tf_distribution"].sample(5).numpy()])

# Step 10: Add a discrete factor
aicon.add_factor_discrete(
    name="ad_count",
    value=3,
    min_value=1,
    max_value=10,
    description="Number of ads per campaign"
)

# Step 11: Verify discrete factor
disc_factor = aicon.brain.get_state_factors()["ad_count"]
print("\nDiscrete factor:")
print(f"Distribution type: {type(disc_factor['tf_distribution']).__name__}")
print(f"5 samples: {disc_factor['tf_distribution'].sample(5).numpy()}")

# Step 12: Show human-readable state
print("\nEntire state:")
print(aicon.get_state(format_nicely=True))

Creating categorical factor with TensorFlow: best_channel
Added categorical factor with TensorFlow distribution: best_channel

Categorical factor:
Distribution type: Categorical
Categories: ['facebook', 'google', 'tiktok', 'instagram']
5 samples: ['google', 'tiktok', 'facebook', 'tiktok', 'google']
Creating discrete factor with TensorFlow: ad_count
Added discrete factor with TensorFlow distribution: ad_count

Discrete factor:
Distribution type: Categorical
5 samples: [2 2 2 2 2]

Entire state:
AIcon State (3 factors):

conversion_rate:
  Type: continuous
  Distribution: normal
  Current value: 0.05
  Mean: 0.05
  Uncertainty: 0.01
  Constraints: >= 0.0, <= 1.0
  Description: Conversion rate for ads

best_channel:
  Type: categorical
  Distribution: categorical
  Current value: facebook
  Categories (probability):
    facebook: 0.40
    google: 0.30
    tiktok: 0.20
    instagram: 0.10
  Description: Best performing ad channel

ad_count:
  Type: discrete
  Distribution: categorical
  Cu

### View all priors

In [5]:
# View all priors
print(aicon.get_state(format_nicely=True))

AIcon State (3 factors):

conversion_rate:
  Type: continuous
  Distribution: normal
  Current value: 0.05
  Mean: 0.05
  Uncertainty: 0.01
  Constraints: >= 0.0, <= 1.0
  Description: Conversion rate for ads

best_channel:
  Type: categorical
  Distribution: categorical
  Current value: facebook
  Categories (probability):
    facebook: 0.40
    google: 0.30
    tiktok: 0.20
    instagram: 0.10
  Description: Best performing ad channel

ad_count:
  Type: discrete
  Distribution: categorical
  Current value: 3
  Possible values: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
  Description: Number of ads per campaign


In [6]:
# Get raw access to all prior distributions
state_factors = aicon.brain.get_state_factors()

# Loop through all factors and show their TF distributions
for name, factor in state_factors.items():
    tf_dist = factor["tf_distribution"]
    print(f"\nPrior for {name}:")
    print(f"  Distribution type: {type(tf_dist).__name__}")
    
    # Sample from the distribution
    samples = tf_dist.sample(1000)
    
    # Show distribution properties
    if hasattr(tf_dist, "mean"):
        print(f"  Mean: {tf_dist.mean().numpy()}")
    if hasattr(tf_dist, "stddev"):
        print(f"  Std Dev: {tf_dist.stddev().numpy()}")
    
    # Show sample statistics
    print(f"  Sample mean: {samples.numpy().mean():.4f}")
    print(f"  Sample stddev: {samples.numpy().std():.4f}")


Prior for conversion_rate:
  Distribution type: TruncatedNormal
  Mean: 0.050000015646219254
  Std Dev: 0.009999962523579597
  Sample mean: 0.0505
  Sample stddev: 0.0098

Prior for best_channel:
  Distribution type: Categorical
  Mean: 1.0
  Std Dev: 1.0
  Sample mean: 0.9410
  Sample stddev: 0.9877

Prior for ad_count:
  Distribution type: Categorical
  Mean: 2.0
  Std Dev: 0.0
  Sample mean: 2.0000
  Sample stddev: 0.0000


### Check Posteriors

In [12]:
# Core imports
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from aicons.definitions.simple_bad_aicon import SimpleBadAIcon
from aicons.bayesbrainGPT.sensors.tf_sensors import MarketingSensor

# Create the AIcon
aicon = SimpleBadAIcon(name="Marketing Campaign")

# Add factors (priors)
aicon.add_factor_continuous(
    name="base_conversion_rate", 
    value=0.05, 
    uncertainty=0.01,
    lower_bound=0.0,
    upper_bound=1.0
)

aicon.add_factor_categorical(
    name="primary_channel",
    value="facebook",
    categories=["facebook", "google", "tiktok", "instagram"],
    probs=[0.4, 0.3, 0.2, 0.1]
)

aicon.add_factor_discrete(
    name="optimal_daily_ads",
    value=5,
    min_value=1,
    max_value=20
)

# Print the prior state
print("PRIOR STATE:")
print(aicon.get_state(format_nicely=True))

# Add a sensor
marketing_sensor = MarketingSensor(name="ad_campaign_data")
aicon.add_sensor("campaign_data", marketing_sensor)
print(f"\nSensor added: {marketing_sensor.name}")
print(f"Observable factors: {marketing_sensor.observable_factors}")
print(f"Factor reliabilities: {marketing_sensor.factor_reliabilities}")

# Define "true" values for testing
true_values = {
    "base_conversion_rate": 0.063,
    "primary_channel": "google",
    "optimal_daily_ads": 8
}
print(f"\nTrue values for testing: {true_values}")

# Update beliefs with sensor data
print("\nUpdating beliefs with sensor data...")
update_result = aicon.update_from_sensor("campaign_data", environment=true_values)
print(f"Update result: {update_result}")

# Get posterior samples with error handling
posterior_samples = aicon.get_posterior_samples()
print("\nPosterior samples keys:", list(posterior_samples.keys()))
print("Number of samples:", {k: len(v) if hasattr(v, '__len__') else 'N/A' for k, v in posterior_samples.items()})

# Print sample statistics for each factor if available
for name, samples in posterior_samples.items():
    if isinstance(samples, np.ndarray):
        if samples.dtype.kind in ['U', 'S']:  # String type (categorical)
            # Get value counts for categorical variables
            values, counts = np.unique(samples, return_counts=True)
            probs = counts / counts.sum()
            print(f"\n{name} posterior:")
            for v, p in zip(values, probs):
                print(f"  {v}: {p:.3f}")
        else:  # Numeric type
            print(f"\n{name} posterior:")
            print(f"  Mean: {np.mean(samples):.4f}")
            print(f"  Std: {np.std(samples):.4f}")
            print(f"  Min: {np.min(samples):.4f}")
            print(f"  Max: {np.max(samples):.4f}")

# Print the posterior state
print("\nPOSTERIOR STATE:")
print(aicon.get_state(format_nicely=True))

Creating continuous factor with TensorFlow: base_conversion_rate
Added continuous factor with TensorFlow distribution: base_conversion_rate
Creating categorical factor with TensorFlow: primary_channel
Added categorical factor with TensorFlow distribution: primary_channel
Creating discrete factor with TensorFlow: optimal_daily_ads
Added discrete factor with TensorFlow distribution: optimal_daily_ads
PRIOR STATE:
AIcon State (3 factors):

base_conversion_rate:
  Type: continuous
  Distribution: normal
  Current value: 0.05
  Mean: 0.05
  Uncertainty: 0.01
  Constraints: >= 0.0, <= 1.0
  Description: Continuous factor: base_conversion_rate

primary_channel:
  Type: categorical
  Distribution: categorical
  Current value: facebook
  Categories (probability):
    facebook: 0.40
    google: 0.30
    tiktok: 0.20
    instagram: 0.10
  Description: Categorical factor: primary_channel

optimal_daily_ads:
  Type: discrete
  Distribution: categorical
  Current value: 5
  Possible values: [1, 2, 3